## Class 모델코드

### 1. 코드 생성 및 검증

In [ ]:
# 방법 1번 - 기몬 모델을 클래스 모델로

# X = tf.keras.Input(shape=[1])
# Y = tf.keras.layers.Dense(1)(X)
# model = tf.keras.Model(X, Y)

import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(1)

    def call(self, inputs):
        return self.dense(inputs)

model = MyModel()
model.compile(loss='mse')




In [ ]:
# 방법 2번 - hidden layers 모델을 클래스 모델로

# X = tf.keras.Input(shape=[1])
# H = tf.keras.layers.Dense(2, activation='swish')(X)
# Y = tf.keras.layers.Dense(1)(H)
# model = tf.keras.Model(X, Y)

import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(2, activation='swish')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, X):
        H = self.dense1(X)
        Y = self.dense2(H)
        return Y

model = MyModel()
model.compile(loss='mse')

In [ ]:
# 방법 3번 - dropout 추가

import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()
model.compile(loss='mse')

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/blackdew/ml-tensorflow/master/data/csv/lemonade.csv")
x_train = df[['온도']]
y_train = df[['판매량']]

In [ ]:
print(model.predict(x_train))
y_train

### 3. fasion model class 생성 실습

In [ ]:
# 원하는 모델!!
# X = tf.keras.Input(shape=[28, 28])
# H = tf.keras.layers.Flatter()(X)
# H = tf.keras.layers.Dense(128, activation='swish')(H)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Dropout(0.3)(H)
# H = tf.keras.layers.Dense(32, activation='swish')(X)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Dropout(0.3)(H)
# Y = tf.keras.layers.Dense(10, activation='softmax')(H)
# model = tf.keras.Model(X, Y)
# model.compile(loss='categorical_crossentropy', metrics='accuracy')
# model.summary()

import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self, **kwargs):
    super(MyModel, self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten()
    
    self.dense1 = tf.keras.layers.Dense(64, activation=tf.nn.swish)
    self.bn1 = tf.keras.layers.BatchNormalization()
    
    self.dense2 = tf.keras.layers.Dense(20, activation=tf.nn.swish)
    self.bn2 = tf.keras.layers.BatchNormalization()
    
    self.dense3 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    

  def call(self, X):
    H = self.flatten(X)
    H = self.dense1(H)
    H = self.bn1(H)
    H = self.dense2(H)
    H = self.bn2(H)
    Y = self.dense3(H)
    return Y

fasion_model = MyModel()
fasion_model.compile(optimizer='adam', loss='spaese_categorical_crossentropy', metrics='accuracy')
fasion_model.build(input_shape=[28, 28])
fasion_model.summary()

In [123]:
# 모델을 준비합니다.
# X = tf.keras.Input(shape=[28, 28])
# H = tf.keras.layers.Flatten()(X)
# H = tf.keras.layers.Dense(64, activation=tf.keras.activations.swish)(H)
# H = tf.keras.layers.BatchNormalization(H)
# H = tf.keras.layers.Dense(20, activation=tf.keras.activations.swish)(H)
# H = tf.keras.layers.BatchNormalization(H)
# Y = tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)(H)
# model = tf.keras.Model(X, Y)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               # loss=tf.keras.losses.sparse_categorical_crossentropy,
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#               # metrics=tf.keras.metrics.sparse_categorical_accuracy,
#               metrics=tf.keras.metrics.SparseCategoricalAccuracy())
# model.summary()

class MyFashionMNISTModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MyFashionMNISTModel, self).__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(64, activation="swish")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.dense2 = tf.keras.layers.Dense(20, activation="swish")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.dense3 = tf.keras.layers.Dense(10, activation="softmax")

    def call(self, X):
        H = self.flatten(X)
        H = self.dense1(H)
        H = self.bn1(H)
        H = self.dropout1(H)
        H = self.dense2(H)
        H = self.bn2(H)
        H = self.dropout2(H)
        Y = self.dense3(H)
        return Y
    
    def train_step(self, batch):
        x_batch, y_batch = batch

        with tf.GradientTape() as tape:
            y_pred = self(x_batch, training=True)
            loss = self.compiled_loss(y_batch, y_pred)

        grad = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad, self.trainable_weights))

        self.compiled_metrics.update_state(y_batch, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, batch):
        x_batch, y_batch = batch
        y_pred = self(x_batch)
        loss = self.compiled_loss(y_batch, y_pred)

        self.compiled_metrics.update_state(y_batch, y_pred)
        return {m.name: m.result() for m in self.metrics}

model = MyFashionMNISTModel()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
model.build(input_shape=[None, 28, 28])
model.summary()

Model: "my_fashion_mnist_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_14 (Flatten)        multiple                  0         
                                                                 
 dense_71 (Dense)            multiple                  50240     
                                                                 
 batch_normalization_28 (Ba  multiple                  256       
 tchNormalization)                                               
                                                                 
 dropout_23 (Dropout)        multiple                  0         
                                                                 
 dense_72 (Dense)            multiple                  1300      
                                                                 
 batch_normalization_29 (Ba  multiple                  80        
 tchNormalization)                        

In [124]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train, x_val = x_train[:48000], x_train[48000:]
y_train, y_val = y_train[:48000], y_train[48000:]

In [125]:
# model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_var, y_var))


early = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True) # 오버피팅이 10번 관측되면 멈추고, 여태 나온 최고 성능으로 모델을 만들어라
result = model.fit(x_train, y_train, epochs=100, batch_size=128,
                   validation_data=(x_val, y_val),
                   callbacks=[early]
)

Epoch 1/100
375/375 [==============================] - 2s 3ms/step - loss: 1.1995 - accuracy: 0.5894 - val_loss: 0.5680 - val_accuracy: 0.8099
Epoch 2/100
375/375 [==============================] - 1s 3ms/step - loss: 0.7875 - accuracy: 0.7295 - val_loss: 0.5031 - val_accuracy: 0.8198
Epoch 3/100
375/375 [==============================] - 1s 3ms/step - loss: 0.6996 - accuracy: 0.7630 - val_loss: 0.4516 - val_accuracy: 0.8374
Epoch 4/100
375/375 [==============================] - 1s 3ms/step - loss: 0.6462 - accuracy: 0.7818 - val_loss: 0.4348 - val_accuracy: 0.8449
Epoch 5/100
375/375 [==============================] - 1s 3ms/step - loss: 0.6222 - accuracy: 0.7904 - val_loss: 0.4484 - val_accuracy: 0.8325
Epoch 6/100
375/375 [==============================] - 1s 3ms/step - loss: 0.5974 - accuracy: 0.8002 - val_loss: 0.4218 - val_accuracy: 0.8480
Epoch 7/100
375/375 [==============================] - 1s 3ms/step - loss: 0.5855 - accuracy: 0.8015 - val_loss: 0.4068 - val_accuracy: 0.8525

## packing / unpacking

In [ ]:
# list의 경우
# packing
a, *b = [1,2,3,4]
print(a, b)

# unpacking
c = [1, b]
d = [1, *b]
print(c)
print(d)

In [ ]:
# dictionary의 경우

# dicionary unpacking
a = {"a": 1, "b": 2, "c": 3}
b = {**a, "d": 4, "e": 5} 
print(b)

# dicionary packing 불가능
a, **b = {"a": 1, "b": 2, "c": 3}

In [ ]:
# 함수에서 사용하는 경우

# 함수에서 list packing
def add(a, b, c):
    return a + b + c

d = [1,2,3]
print(add(d[0],d[1],d[2])) # 우리가 packing을 몰랐다면....
print(add(*d)) # packing을 아니까 이렇게 편하다

print("-"*50)

# 함수에서 dictionary packing
def add(**a):
    print(a)
    return sum(a.values())

print(add(a=1))
print(add(a=1, b=2, c=3))

In [ ]:
# packing
def add(*a):
    print(a)
    return sum(a)

print(add(1))
print(add(1, 2))
print(add(1, 2, 3))
print(add(1, 2, 3, 4))